In [1]:
import numpy as np
import torch
import pandas as pd

In [2]:
df = pd.read_csv("manual_data.csv")

In [3]:
df.head()

,waterfall_filename,time_start,time_end,channel_start,channel_end,vehicle_type,uphill
0,sensor_2024-11-24T070135-0800.h5_Start0.0_End30.0,22.5,30.0,153,79,large,1
1,sensor_2024-11-24T070135-0800.h5_Start30.0_End...,30.0,40.0,79,0,large,1
2,sensor_2024-11-24T084635-0800.h5_Start0.0_End30.0,5.0,18.0,153,0,small,1
3,sensor_2024-11-24T084635-0800.h5_Start0.0_End30.0,16.5,30.0,153,10,large,1
4,sensor_2024-11-24T093235-0800.h5_Start30.0_End...,41.0,53.0,153,0,small,1


In [4]:
df = df[df.uphill == 1]

In [5]:
len(df)

41

In [36]:
imgs = []
labels = []
target_shape = (585, 153)
for index, rows in df.iterrows():
    f = rows["waterfall_filename"]
    label = rows["vehicle_type"]
    new_name = "DAS_data/FBE_Sensor"
    f = f.replace("sensor", "")
    f = f.replace(".h5", "")
    f = f.replace("Start", "")
    f = f.replace("End", "")
    f = f.replace("0.0_30.0", "0s-30s")
    f = f.replace("30.0_60.0", "30s-60s")
    f = new_name + f + "_10-200Hz.npy"
    imgs.append(np.load(f)[:585, :153])
    labels.append(label)

In [37]:
imgs = np.asarray(imgs)
print(imgs.shape)

(41, 585, 153)


In [38]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, imgs, labels, transforms, noise = 1e-3):
        self.imgs = imgs
        self.labels = labels
        self.transforms = transforms
        self.noise = noise
    
    def __getitem__(self, i):
        return (self.transforms(self.imgs[i]) + torch.randn(self.imgs[i].shape)*self.noise, self.labels[i])
    
    def __len__(self):
        return len(self.imgs)

In [39]:
label_map = {0: "small", 1: "large"}
for i in range(len(labels)):
    if labels[i] == "small":
        labels[i] = 0
    elif labels[i] == "large":
        labels[i] = 1


In [40]:
labels = np.asarray(labels)
print(labels)

[1 1 0 1 0 0 0 1 0 1 0 1 1 1 0 1 1 0 1 1 0 0 1 1 0 1 1 1 0 0 0 1 0 1 1 0 0
 0 0 0 0]


In [11]:
from torchvision.transforms import v2

In [12]:
transforms = v2.Compose([
    v2.RandomHorizontalFlip(p=0.5),
    v2.RandomVerticalFlip(p=0.5),
    v2.Normalize((0.5, ), (0.5, )),
    v2.ToDtype(torch.float32, scale=True),
])

In [13]:
train_imgs = imgs[5:]
train_labels = labels[5:]
test_imgs = imgs[:5]
test_labels = labels[:5]

In [14]:
train_imgs = torch.from_numpy(train_imgs)

In [15]:
train_imgs = train_imgs.reshape((36, 1, 585, 153))
print(train_imgs.shape)

torch.Size([36, 1, 585, 153])


In [25]:
train_ds = Dataset(train_imgs, train_labels, transforms)
test_ds = Dataset(test_imgs, test_labels, transforms, noise = 0)

In [17]:
train_labels

array([0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1,
       1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0])

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class SimpleResNet(nn.Module):
    def __init__(self):
        super(SimpleResNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)

        self.layer1 = BasicBlock(32, 64, stride=2)
        self.layer2 = BasicBlock(64, 128, stride=2)
        self.layer3 = BasicBlock(128, 256, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(256, 1)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avgpool(out)
        out = torch.flatten(out, 1)
        out = self.fc(out)
        return out


In [19]:
model = SimpleResNet()
out = model(torch.zeros((1, 1, 585, 153)))

In [24]:
from torch.utils.data import DataLoader
import torch.optim as optim
from tqdm import tqdm

# Assume model is defined and called 'model'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Dummy placeholders - replace with your real dataset
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)

# Loss and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Training loop
num_epochs = 10
for epoch in tqdm(range(num_epochs)):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.float().to(device).unsqueeze(1)  # Ensure shape [B, 1]
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        preds = torch.sigmoid(outputs) > 0.5
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_acc = correct / total
    avg_loss = running_loss / len(train_loader)

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}, Accuracy: {train_acc:.4f}")


 10%|████▍                                       | 1/10 [00:21<03:14, 21.60s/it]

Epoch 1/10, Loss: 0.7544, Accuracy: 0.6944


 20%|████████▊                                   | 2/10 [00:45<03:04, 23.10s/it]

Epoch 2/10, Loss: 0.5890, Accuracy: 0.6944


 30%|█████████████▏                              | 3/10 [01:07<02:38, 22.71s/it]

...

Epoch 8/10, Loss: 0.5799, Accuracy: 0.7500


 90%|███████████████████████████████████████▌    | 9/10 [03:29<00:25, 25.10s/it]

Epoch 9/10, Loss: 0.5949, Accuracy: 0.6389


100%|███████████████████████████████████████████| 10/10 [03:55<00:00, 23.60s/it]

Epoch 10/10, Loss: 0.6387, Accuracy: 0.6944


In [27]:
imgs

array([[[-83.16533673, -80.48941362, -80.82026303, ..., -86.40004322,
         -79.86596284, -82.38288391],
        [-81.44822938, -80.90790602, -78.95710824, ..., -85.29818789,
         -83.51177394, -81.44601834],
        [-83.35952402, -78.41031449, -81.84194271, ..., -86.13143581,
         -81.52151458, -80.6781823 ],
        ...,
        [-77.83159733, -80.48285566, -85.15926687, ..., -86.48475857,
         -79.98943363, -82.98869435],
        [-80.82277576, -79.08962964, -80.26693421, ..., -86.61039554,
         -84.07328293, -83.1402916 ],
        [-78.30538092, -73.51581599, -77.53454542, ..., -82.98654912,
         -80.07903555, -82.23120977]],

       [[-78.18900907, -78.6662453 , -78.81660319, ..., -86.67769961,
         -76.21459263, -81.8938268 ],
        [-81.22421343, -82.4349877 , -81.69241862, ..., -87.19092394,
         -74.75361206, -80.00945675],
        [-86.12481637, -77.3260291 , -79.61768577, ..., -87.61009091,
         -78.31058595, -83.94645948],
        ...,


In [41]:
np.save("images.npy", imgs)
np.save("labels.npy", labels)

In [42]:
labels

array([1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0,
       1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0])